## MVO Weights
4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,
2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,
7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245


In [2]:
import pandas as pd
import numpy as np
import random
import math
from datetime import date,timedelta

In [3]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights

In [4]:
def ratio(a,b,c):                       
  #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  #calculating sharpe ratio
  return (a-c)/b

In [5]:
# def number_of_years(y):#calculates the number of years of the dataset
#   p=y.index[0]         #date of first row in the dataset (datetime format)
#   q=y.index[len(y)-1]  #date of last row in the dataset  (datetime format)
#   return ((q-p).days+1)/365

In [6]:
df=pd.read_csv("2021-01-06.csv",parse_dates=['Date'],index_col='Date')  #Importing Dataset
#df = df.loc["2016-01-01" :"2020-12-31" ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)
col=list(df.columns)
tdf.shape

(3, 24)

In [7]:
end=tdf.index[2].date()

In [8]:
trading_days=250#len(df)/number_of_years(tdf) 

In [9]:
returnsh=df.pct_change()            
#Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)

In [10]:
returnsh=returnsh.iloc[1:,:]
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,0.014580,-0.018634,-0.008754,-0.000507,-0.005217,-0.003266,0.011169,0.007556,-0.015729,0.009911,...,0.009811,-0.002388,-0.012432,0.017618,0.009189,-0.017965,0.001896,0.012928,0.002628,0.024975
2,0.004116,-0.017328,-0.003911,-0.003210,0.009825,0.024238,-0.013265,-0.004311,0.034958,-0.013568,...,-0.002317,0.043351,-0.026372,-0.013417,-0.007547,0.004776,-0.006674,0.001050,0.020061,0.000246


In [11]:
returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
  for i in range(1, len(df)):
    if returnso[cols][i] > 0:
      returnso[cols][i] = 0


In [12]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,0.0,-0.018634,-0.008754,-0.000507,-0.005217,-0.003266,0.000000,0.000000,-0.015729,0.000000,...,0.000000,-0.002388,-0.012432,0.000000,0.000000,-0.017965,0.000000,0.0,0.0,0.0
2,0.0,-0.017328,-0.003911,-0.003210,0.000000,0.000000,-0.013265,-0.004311,0.000000,-0.013568,...,-0.002317,0.000000,-0.026372,-0.013417,-0.007547,0.000000,-0.006674,0.0,0.0,0.0


In [13]:
covmatsh=returnsh.cov()*trading_days     
#Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso = returnso.cov() * trading_days

In [14]:
covmatsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
ASIANPAINT,0.013688,-0.001709,-0.006335,0.003537,-0.019676,-0.035977,0.031960,0.015523,-0.066301,0.030713,...,0.015864,-0.059828,0.018234,0.040596,0.021890,-0.029746,0.011211,0.015536,-0.022803,0.032346
BAJFINANCE,-0.001709,0.000213,0.000791,-0.000442,0.002457,0.004492,-0.003991,-0.001938,0.008279,-0.003835,...,-0.001981,0.007471,-0.002277,-0.005069,-0.002733,0.003714,-0.001400,-0.001940,0.002847,-0.004039
BAJAJFINSV,-0.006335,0.000791,0.002932,-0.001637,0.009106,0.016650,-0.014791,-0.007184,0.030685,-0.014214,...,-0.007342,0.027689,-0.008439,-0.018788,-0.010131,0.013767,-0.005188,-0.007190,0.010554,-0.014970
BRITANNIA,0.003537,-0.000442,-0.001637,0.000914,-0.005084,-0.009296,0.008258,0.004011,-0.017131,0.007935,...,0.004099,-0.015458,0.004711,0.010489,0.005656,-0.007686,0.002897,0.004014,-0.005892,0.008358
DIVISLAB,-0.019676,0.002457,0.009106,-0.005084,0.028284,0.051716,-0.045942,-0.022314,0.095308,-0.044149,...,-0.022804,0.086003,-0.026212,-0.058356,-0.031467,0.042760,-0.016115,-0.022334,0.032780,-0.046497
GRASIM,-0.035977,0.004492,0.016650,-0.009296,0.051716,0.094560,-0.084003,-0.040799,0.174264,-0.080724,...,-0.041695,0.157252,-0.047926,-0.106701,-0.057536,0.078184,-0.029466,-0.040836,0.059935,-0.085018
HCLTECH,0.031960,-0.003991,-0.014791,0.008258,-0.045942,-0.084003,0.074624,0.036244,-0.154808,0.071712,...,0.037040,-0.139695,0.042576,0.094788,0.051113,-0.069455,0.026176,0.036277,-0.053244,0.075526
HDFCBANK,0.015523,-0.001938,-0.007184,0.004011,-0.022314,-0.040799,0.036244,0.017603,-0.075188,0.034829,...,0.017990,-0.067848,0.020678,0.046037,0.024825,-0.033733,0.012713,0.017619,-0.025860,0.036682
HINDALCO,-0.066301,0.008279,0.030685,-0.017131,0.095308,0.174264,-0.154808,-0.075188,0.321150,-0.148766,...,-0.076840,0.289798,-0.088323,-0.196638,-0.106033,0.144084,-0.054302,-0.075256,0.110455,-0.156678
HINDUNILVR,0.030713,-0.003835,-0.014214,0.007935,-0.044149,-0.080724,0.071712,0.034829,-0.148766,0.068913,...,0.035595,-0.134243,0.040914,0.091089,0.049118,-0.066744,0.025154,0.034861,-0.051166,0.072578


In [15]:
risk_free_rate = 0.0358 #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [16]:
df
stocks=df.shape[1]
stocks

24

# Sharpe


In [17]:
global_warr_sortino=[]
global_war_sharpe=[]
sharpe_portfolio_return=[]
sharpe_portfolio_risk=[]
sharpe_portfolio_shratio=[]
sharpe_portfolio_stockWeights=[]

In [18]:

fitness=0
#for each iteration

weights=np.array(BALANCE(np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varsh=np.dot(weights.T,np.dot(covmatsh,weights))   

#portfolio risk
volatility_temp = np.sqrt(varsh)      

#Calculating fitness value(ie sharpe ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sharpe_pbest)


sharpe_portfolio_return.append(returns_temp)
sharpe_portfolio_risk.append(volatility_temp)
sharpe_portfolio_shratio.append(fitness)
sharpe_portfolio_stockWeights.append(weights)



In [19]:
sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  

for counter,symbol in enumerate(df.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)
sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
sharpe_optimal=pd.DataFrame(sharpe_optimal)
sharpe_optimal.to_csv("sharpe_optimal.csv")

sharpe_pc

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,-0.305383,0.037709,-9.047791,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [20]:
sharpe_pc.to_csv(str(end)+'MVOsharpe_A.csv')

# Sortino

In [21]:
global_warr_sortino=[]
sortino_portfolio_return=[]
sortino_portfolio_risk=[]
sortino_portfolio_soratio=[]
sortino_portfolio_stockWeights=[]

In [22]:

fitness=0
#for each iteration


#Making sure that the total amount of pheromone equals 1 
weights=np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varso=np.dot(weights.T,np.dot(covmatso,weights))   

#portfolio risk
volatility_temp = np.sqrt(varso)      

#Calculating fitness value(ie sortino ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sortino_pbest)

sortino_portfolio_return.append(returns_temp)
sortino_portfolio_risk.append(volatility_temp)
sortino_portfolio_soratio.append(fitness)
sortino_portfolio_stockWeights.append(weights)


In [23]:
sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  

for counter,symbol in enumerate(df.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
sortino_pc = pd.DataFrame(sortino_portfolio)
sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
sortino_optimal=pd.DataFrame(sortino_optimal)
sortino_optimal.to_csv("sortino_optimal.csv")
sortino_pc


,Returns,Semi Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,-30.538335,1.763347,-17.338692,4.576064,9.676986,2.881647,3.762807,6.100055,1.147082,1.639964,...,9.762288,2.014192,7.839004,7.728703,2.277867,3.332264,1.031637,7.217097,0.606272,1.736266


In [24]:

sortino_pc.to_csv(str(end)+'MVOsortino_A.csv')